# GLOBSEC

MariaDB

51.4 GB

How to work with MariaDB:
- https://www.a2hosting.com/kb/developer-corner/mysql/managing-mysql-databases-and-users-from-the-command-line

In [1]:
%matplotlib inline

import os
import pandas as pd
import math
import re
import seaborn as sns
import numpy as np
import glob
from functools import reduce

from matplotlib import pyplot as plt
import matplotlib.dates as md
from datetime import datetime,timedelta

In [2]:
import matplotlib

plt.rcParams['figure.figsize'] = 50, 10

font = {'size'   : 30}

matplotlib.rc('font', **font)

# Connect

In [1]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="db",
    database="globs_all"
)

mycursor = mydb.cursor()
print(mydb)

In [2]:
# executing queries
def eq(q):
#     mycursor = mydb.cursor()
    mycursor.execute(q)
    myresult = mycursor.fetchall()

    for x in myresult:
        print(x)

# Tables

Source - RSS

In [3]:
eq('SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA="globs_all"')

('article',)
('article_category',)
('article_category_article',)
('article_label',)
('label',)
('label_article_label',)
('label_category',)
('site',)


# Basic overview

## Article

stiahnuté články v HTML aj textovej podobe


- id
- public_id


- site_id = site


- title
- author
- url


- language
- state


- body
- original_body
- text_body


- date_created
- date_added


BASIC INFO (1st batch)
- n of articles = 30 4537 (36 424)
- missing author = 293 271 (25 517)
- missing date_created = 14 496 (6 853)
- 128 sources (sites) (126)

In [4]:
# how many articles
q = "SELECT COUNT(*) FROM article"
eq(q)

(304537,)


In [5]:
# missing attributes

print('\nmissing author')
q = "SELECT COUNT(*) FROM article WHERE author IS NULL"
eq(q)

print('\nmissing date_created')
q = "SELECT COUNT(*) FROM article WHERE date_created IS NULL"
eq(q)


missing author
(293271,)

missing date_created
(14496,)


In [8]:
# how many 

# sites
print('\nnumber of sites')
q = "SELECT COUNT(DISTINCT (site_id)) FROM article"
eq(q)

# languages
print('\nnumber of languages')
q = "SELECT COUNT(DISTINCT (language)) FROM article"
eq(q)

q = "SELECT language, COUNT(*) FROM article GROUP BY language"
eq(q)

# state
print('\nnumber of states')
q = "SELECT COUNT(DISTINCT (state)) FROM article"
eq(q)

q = "SELECT state, COUNT(*) FROM article GROUP BY state"
eq(q)


number of sites
(128,)

number of languages
(9,)
('bg', 24925)
('cz', 23582)
('hu', 32576)
('me', 29376)
('mk', 49220)
('ro', 40476)
('rs', 39272)
('sk', 64454)
('sr', 656)

number of states
(5,)
('grabbed', 283352)
('labeled', 1490)
('partially_labeled', 239)
('reprocess', 35)
('unrelated', 19421)


In [9]:
# date_created range - however, not every date is properly formatted
print('date_created range')
q = "SELECT YEAR(date_created), COUNT(*) FROM article GROUP BY YEAR(date_created) ORDER BY YEAR(date_created) ASC"
eq(q)

date_created range
(None, 14496)
(0, 121)
(400, 172)
(800, 150)
(1200, 114)
(1600, 144)
(2000, 146)
(2012, 1)
(2013, 568)
(2014, 892)
(2015, 1172)
(2016, 1238)
(2017, 12216)
(2018, 79656)
(2019, 190638)
(2400, 181)
(2800, 164)
(3200, 146)
(3600, 110)
(4000, 139)
(4400, 203)
(4800, 183)
(5200, 161)
(5600, 101)
(6000, 203)
(6400, 153)
(6800, 118)
(7200, 111)
(7600, 151)
(8000, 139)
(8400, 140)
(8800, 97)
(9200, 178)
(9600, 135)


In [10]:
# date_added range - however, not every date is properly formatted
print('date_added range')
q = "SELECT YEAR(date_added), COUNT(*) FROM article GROUP BY YEAR(date_added) ORDER BY YEAR(date_added) ASC"
eq(q)

date_added range
(2019, 304537)


In [11]:
print('number of articles per site (id) - max and min')

print('\nMin - Site ID, Number of articles')
q = "SELECT site_id, COUNT(*) AS c FROM article GROUP BY site_id ORDER BY COUNT(*) ASC LIMIT 5"
eq(q)

print('\nMax - Site ID, Number of articles')
q = "SELECT site_id, COUNT(*) AS c FROM article GROUP BY site_id ORDER BY COUNT(*) DESC LIMIT 5"
eq(q)

number of articles per site (id) - max and min

Min - Site ID, Number of articles
(47, 1)
(112, 10)
(81, 10)
(12, 10)
(61, 11)

Max - Site ID, Number of articles
(2, 47974)
(56, 26608)
(86, 26482)
(135, 16793)
(108, 16171)


In [12]:
# number of articles per site
print('histogram - number of articles per a site')
print('\nNumber of articles , Number of sites')

q = "SELECT c, SUM(c) FROM (SELECT site_id, COUNT(*) AS c FROM article GROUP BY site_id) AS t GROUP BY c ORDER BY c ASC"
eq(q)

histogram - number of articles per a site

Number of articles , Number of sites
(1, Decimal('1'))
(10, Decimal('30'))
(11, Decimal('11'))
(13, Decimal('26'))
(16, Decimal('16'))
(19, Decimal('19'))
(23, Decimal('23'))
(30, Decimal('60'))
(32, Decimal('32'))
(48, Decimal('48'))
(49, Decimal('49'))
(51, Decimal('51'))
(52, Decimal('52'))
(61, Decimal('61'))
(62, Decimal('62'))
(69, Decimal('69'))
(70, Decimal('70'))
(72, Decimal('72'))
(90, Decimal('90'))
(91, Decimal('91'))
(97, Decimal('97'))
(98, Decimal('98'))
(101, Decimal('101'))
(104, Decimal('104'))
(106, Decimal('212'))
(107, Decimal('107'))
(121, Decimal('242'))
(126, Decimal('126'))
(127, Decimal('127'))
(128, Decimal('128'))
(130, Decimal('130'))
(132, Decimal('132'))
(135, Decimal('135'))
(137, Decimal('137'))
(139, Decimal('139'))
(147, Decimal('147'))
(156, Decimal('156'))
(163, Decimal('163'))
(169, Decimal('169'))
(170, Decimal('170'))
(185, Decimal('185'))
(186, Decimal('186'))
(187, Decimal('374'))
(188, Decimal('188')

## Article category

pokiaľ bol článok sťahovaný cez RSS a v RSS webu boli kategórie, tu sú uložené

- id


- site_id = site


- name
- excluded

In [13]:
print('number of categories: \nOriginally: 26 365 \nNow:')

q = "SELECT COUNT(*) FROM article_category"
eq(q)

print('\nexamples (id, name, site_id, excluded):')
q = "SELECT * FROM article_category LIMIT 5"
eq(q)

number of categories: 
Originally: 26 365 
Now:
(23083,)

examples (id, name, site_id, excluded):
(13136, 'Ekonomika', 77, 0)
(13137, 'Ekonomika', 2, 0)
(13138, 'Ekonomika', 83, 0)
(13139, 'Ekonomika', 84, 0)
(13140, 'Ekonomika', 11, 0)


In [14]:
print('number of unique categories: \nOriginally: 12 372 \nNow:')

q = "SELECT COUNT(*) FROM article_category"
eq(q)

number of unique categories: 
Originally: 12 372 
Now:
(23083,)


In [19]:
# missing attributes
print('what s missing - site')

q = "SELECT COUNT(*) FROM article_category WHERE site_id IS NULL"
eq(q)
q = "SELECT COUNT(*) FROM article_category WHERE name IS NULL"
eq(q)
q = "SELECT COUNT(*) FROM article_category WHERE excluded IS NULL"
eq(q)

what s missing - site
(0,)
(0,)
(0,)


In [29]:
# number of categories per site

print('sites with categories - number of categories: site_id, count of their categories')
q = "SELECT site_id, COUNT(*) FROM article_category GROUP BY site_id  ORDER BY COUNT(*) DESC"
eq(q)

sites with categories - number of categories: site_id, count of their categories
(123, 2312)
(134, 2261)
(101, 1703)
(57, 1495)
(115, 1398)
(46, 1314)
(127, 1209)
(79, 773)
(71, 758)
(118, 736)
(109, 718)
(84, 693)
(107, 689)
(49, 649)
(59, 620)
(94, 617)
(138, 566)
(11, 470)
(100, 441)
(70, 439)
(108, 418)
(89, 280)
(60, 280)
(33, 261)
(51, 132)
(61, 132)
(43, 117)
(145, 112)
(150, 92)
(77, 74)
(85, 72)
(55, 70)
(151, 70)
(144, 59)
(91, 58)
(28, 54)
(82, 50)
(93, 49)
(98, 48)
(3, 43)
(97, 43)
(29, 40)
(56, 39)
(148, 39)
(87, 38)
(4, 36)
(103, 35)
(76, 35)
(157, 32)
(132, 32)
(120, 28)
(65, 28)
(45, 28)
(136, 23)
(92, 21)
(110, 21)
(44, 21)
(31, 20)
(152, 20)
(137, 16)
(113, 14)
(105, 13)
(2, 12)
(147, 12)
(30, 11)
(39, 11)
(126, 10)
(83, 9)
(12, 9)
(121, 8)
(140, 7)
(74, 6)
(42, 6)
(112, 6)
(27, 5)
(81, 4)
(111, 4)
(146, 3)
(99, 3)
(142, 1)
(95, 1)
(14, 1)


## Article category article

prepojovacia tabuľka article a article_category (many-to-many relation)


- article_category_id = article_category_id
- article_id = article_id

In [30]:
print('how many mappings: \nOriginally: 40 931 \nNow:')

q = "SELECT COUNT(*) FROM article_category_article"
eq(q)

how many mappings: 
Originally: 40 931 
Now:
(46201,)


In [35]:
# how many categories per article

print('articles with categories: \nOriginally: 18 572 \nNow:')
# q = "SELECT article_id, COUNT(*) FROM article_category_article GROUP BY article_id  ORDER BY COUNT(*) DESC"
q = "SELECT COUNT(*) FROM (SELECT COUNT(*) FROM article_category_article GROUP BY article_id) as t"
eq(q)

articles with categories: 
Originally: 18 572 
Now:
(22253,)


## Article label

labely v článkoch vytvorené používateľmi


- id


- article_id = article


- user_id
- date_created
- path


- content
- parent_content
- offset
- lenght
- html_label



In [36]:
# number of (unique) article label
print('number of article_label:  \nOriginally: 3 572 \nNow:')
q = "SELECT COUNT(*) FROM article_label"
eq(q)

number of article_label:  
Originally: 3 572 
Now:
(7180,)


In [37]:
# users making labels
print('users, how many labels they made')
q = "SELECT user_id, COUNT(*) FROM article_label GROUP BY user_id ORDER BY COUNT(*)"
eq(q)

users, how many labels they made
(1, 1)
(12, 18)
(5, 65)
(3, 89)
(31, 98)
(27, 129)
(20, 154)
(9, 180)
(18, 202)
(6, 302)
(7, 303)
(15, 345)
(30, 462)
(21, 466)
(10, 480)
(8, 500)
(28, 529)
(26, 550)
(19, 581)
(29, 693)
(4, 1033)


In [38]:
# how many labels per article
print('how many labels per article')
q = "SELECT article_id, COUNT(*) FROM article_label GROUP BY article_id ORDER BY COUNT(*)"
eq(q)

how many labels per article
(342136, 1)
(432956, 1)
(436561, 1)
(442445, 1)
(443846, 1)
(446075, 1)
(458955, 1)
(470522, 1)
(473942, 1)
(202351, 1)
(211352, 1)
(242227, 1)
(256179, 1)
(294934, 1)
(305379, 1)
(393444, 1)
(437165, 1)
(456959, 1)
(459210, 1)
(461025, 1)
(231996, 1)
(235758, 1)
(247553, 1)
(291535, 1)
(298055, 1)
(395715, 1)
(404923, 1)
(431320, 1)
(436282, 1)
(458021, 1)
(461369, 1)
(464483, 1)
(179808, 1)
(197772, 1)
(208963, 1)
(233854, 1)
(236758, 1)
(250309, 1)
(279565, 1)
(299511, 1)
(303353, 1)
(391666, 1)
(434725, 1)
(459001, 1)
(460192, 1)
(470550, 1)
(221768, 1)
(235698, 1)
(243156, 1)
(256370, 1)
(295364, 1)
(300397, 1)
(332105, 1)
(393687, 1)
(443241, 1)
(449766, 1)
(459223, 1)
(462442, 1)
(470914, 1)
(184717, 1)
(192787, 1)
(224423, 1)
(232020, 1)
(247569, 1)
(262041, 1)
(298069, 1)
(302011, 1)
(314812, 1)
(431331, 1)
(436321, 1)
(445439, 1)
(447660, 1)
(455286, 1)
(458106, 1)
(461375, 1)
(464535, 1)
(473026, 1)
(179841, 1)
(199171, 1)
(228501, 1)
(237725, 1)


(305511, 5)
(313523, 5)
(176719, 5)
(298773, 5)
(316839, 5)
(383828, 5)
(176922, 5)
(235138, 5)
(293992, 5)
(303660, 5)
(320305, 5)
(434835, 5)
(438776, 5)
(450601, 5)
(456907, 5)
(459091, 5)
(234781, 5)
(323208, 5)
(355537, 5)
(439645, 5)
(457806, 5)
(179032, 5)
(185453, 5)
(235842, 5)
(310919, 5)
(317090, 5)
(341413, 5)
(376123, 5)
(458311, 5)
(303809, 5)
(379373, 5)
(450624, 5)
(456916, 5)
(184365, 5)
(234421, 5)
(234794, 5)
(235299, 5)
(284436, 5)
(323656, 5)
(443440, 5)
(304860, 5)
(448383, 5)
(462883, 5)
(294228, 5)
(353942, 5)
(466158, 5)
(304511, 5)
(381652, 5)
(450018, 5)
(463659, 5)
(179415, 5)
(235547, 5)
(278674, 5)
(299255, 5)
(303063, 5)
(306746, 5)
(438221, 5)
(440240, 5)
(210088, 5)
(379573, 5)
(234458, 5)
(268659, 5)
(301492, 5)
(310251, 5)
(323910, 5)
(430941, 5)
(439906, 5)
(447499, 5)
(450020, 5)
(461330, 5)
(234569, 5)
(377125, 5)
(460081, 5)
(186906, 5)
(202336, 5)
(331481, 5)
(374006, 5)
(450898, 5)
(234483, 5)
(268781, 5)
(334840, 5)
(443445, 5)
(321406, 6)
(354

## Label

definícia labelu podľa administrátorov globsecu

 tri tematicke labely (3 Russia as a protector of traditional values, 1 USA as manipulator and devious actor, 2 EU dictate), labely definujuce krajiny, a dalsi label ak sa nachadza tematicky label priamo v nadpise (Headline)
 
- id


- category_id = category


- created_by
- name
- description
- created_date

In [39]:
print('number of labels: \nOriginally: 161 \nNow:')

q = "SELECT COUNT(*) FROM label"
eq(q)

number of labels: 
Originally: 161 
Now:
(162,)


## Label article label

prepojovacia tabuľka tabuľky label a tabuľky article_label (many-to-many relation)


- label_id = label
- article_label_id = article_label_id

In [40]:
print('number of mappings: \nOriginally: 2 303 \nNow:')

q = "SELECT COUNT(*) FROM label_article_label"
eq(q)

number of mappings: 
Originally: 2 303 
Now:
(7872,)


## Label category

všetky labely sú v systéme rozdelené do kategórii podľa toho aké tematické okruhy si globsec administrátori určili

- id


- name
- description
- created_date
- members_can_add_labels
- color

v zasade = len farby

In [41]:
print('number of: \nOriginally: 5 \nNow:')

q = "SELECT COUNT(*) FROM label_category"
eq(q)

q = "SELECT * FROM label_category"
eq(q)

number of: 
Originally: 5 
Now:
(5,)
(5, 'Russia', None, datetime.datetime(2019, 5, 5, 18, 58, 17), 0, 'red')
(6, 'Specific tags', None, datetime.datetime(2019, 5, 5, 18, 58, 51), 0, 'blue')
(8, 'USA', None, datetime.datetime(2019, 5, 15, 12, 59, 11), 0, 'orange')
(9, 'EU', None, datetime.datetime(2019, 5, 15, 13, 3, 8), 0, 'pink')
(11, 'List of countries', None, datetime.datetime(2019, 5, 27, 14, 45, 30), 0, 'grey')


## Site

stránky odkiaľ sú články stiahnuté


- id


- name
- enabled - ???
- host
- language
- proto
- slug
- type
- last_scraped
- last_processed

In [42]:
print('number of: \nOriginally: 144 \nNow:')

q = "SELECT COUNT(*) FROM site"
eq(q)

number of: 
Originally: 144 
Now:
(150,)


In [44]:
print('what s missing: last_scraped and last_processed')

q = "SELECT COUNT(*) FROM site WHERE last_scraped IS NULL"
eq(q)
q = "SELECT COUNT(*) FROM site WHERE last_processed IS NULL"
eq(q)

what s missing: last_scraped and last_processed
(17,)
(150,)


In [45]:
print('how many sites per language:')
q = "SELECT language, COUNT(*) FROM site GROUP BY language ORDER BY COUNT(*) DESC"
eq(q)

how many sites per language:
('sk', 28)
('ro', 22)
('bg', 20)
('rs', 19)
('cz', 17)
('hu', 17)
('mk', 14)
('me', 13)


In [47]:
print('how many sites per type: (previously 68,30,23,23)')
q = "SELECT type, COUNT(*) FROM site GROUP BY type ORDER BY COUNT(*) DESC"
eq(q)

how many sites per type: (previously 68,30,23,23)
('conspi', 74)
('hyper_partisan', 30)
('balanced', 23)
('grey', 23)


# All-frame

number of articles per type

In [68]:
# number of articles per type
print('how many articles per type:')

q = "SELECT site.type, COUNT(*) FROM article LEFT JOIN site ON article.site_id=site.id GROUP BY site.type"
eq(q)

how many articles per type:
('balanced', 21879)
('conspi', 145476)
('grey', 55748)
('hyper_partisan', 81434)


## Labels - languages, sites, types...

### how many labels per language

In [69]:
# how many labels per language
print('how many labels per language:')

q = "SELECT article.language, COUNT(*) FROM article INNER JOIN article_label ON article.id=article_label.article_id GROUP BY article.language"
eq(q)

how many labels per language:
('bg', 1305)
('cz', 1033)
('hu', 1109)
('me', 382)
('mk', 500)
('ro', 432)
('rs', 1313)
('sk', 1106)


## Labels

In [73]:
# how many labels categories
print('how many labels per categories:')

q = "SELECT label.name, COUNT(*) FROM label_article_label INNER JOIN label ON label_article_label.label_id=label.id GROUP BY label.name"
eq(q)

how many labels per categories:
('1 USA as manipulator and devious actor', 2921)
('2 EU dictate', 1264)
('3 Russia as a protector of traditional values', 237)
('4 Russia as a military superpower', 367)
('Afghanistan', 18)
('Albania', 14)
('Algeria', 1)
('Angola', 1)
('Argentina', 2)
('Armenia', 1)
('Australia', 5)
('Austria', 6)
('Azerbaijan', 2)
('Belarus', 1)
('Belgium', 4)
('Bolivia', 2)
('Bosnia and Herzegovina', 26)
('Brazil', 5)
('Bulgaria', 32)
('Canada', 5)
('Chad', 1)
('Chile', 4)
('China', 93)
('Colombia', 8)
('Congo, Democratic Republic of the', 1)
('Costa Rica', 1)
('Croatia', 9)
('Cuba', 12)
('Cyprus', 3)
('Czechia', 19)
('Denmark', 5)
('Ecuador', 3)
('Egypt', 5)
('El Salvador', 1)
('Estonia', 5)
('Ethiopia', 1)
('EU', 207)
('Finland', 3)
('France', 75)
('Georgia', 7)
('Germany', 81)
('Greece', 13)
('Guatemala', 1)
('Haiti', 1)
('Headline', 730)
('Honduras', 1)
('Hungary', 39)
('India', 8)
('Iran', 68)
('Iraq', 20)
('Ireland', 3)
('Israel', 23)
('Italy', 27)
('Japan', 8)
(